In [1]:
import numpy as np
import pandas as pd
import warnings
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import preprocessing

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./train.csv')
#df_test = pd.read_csv('./test.csv')

In [3]:
df = df.drop(['Name','Ticket'],axis = 1)
X = pd.DataFrame(df)
X = X.drop('Survived',axis = 1)
Y = df['Survived']
X.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Data Processing
 - Missing values in age replaced with average of age
 - Missing values in embarked replaced with 'U'
 - Missing Values in Cabin replaced with 'S'
 - Sex, Emabarked, and Cabin transformed using label encoder

In [4]:
# Function for normalizing data in a column given the dataframe, and column name
def normalize_col(df,col_name):
    Temp_Col = pd.DataFrame(df[col_name])
    col_normal = preprocessing.MinMaxScaler().fit_transform(Temp_Col.copy())
    col_normal = pd.DataFrame(col_normal, columns = Temp_Col.columns)
    df[col_name] = col_normal
    return df

In [5]:
ave_age = round(X['Age'].mean(),2)
X['Age'] = X['Age'].replace(np.nan, ave_age ,regex = True)
X['Embarked'] = X['Embarked'].replace(np.nan, 'S',regex = True)
X['Cabin'] = X['Cabin'].replace(np.nan, 'U',regex = True)
feature_for_encoding = ['Sex','Embarked','Cabin']

encoder = LabelEncoder()
for feature in feature_for_encoding:
    X[feature] = encoder.fit_transform(X[feature])
    
    
print(X.info())
print(X.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    int32  
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Cabin        891 non-null    int32  
 8   Embarked     891 non-null    int32  
dtypes: float64(2), int32(3), int64(4)
memory usage: 52.3 KB
None
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Cabin  Embarked
0            1       3    1  22.0      1      0   7.2500    147         2
1            2       1    0  38.0      1      0  71.2833     81         0
2            3       3    0  26.0      0      0   7.9250    147         2
3            4       1    0  35.0      1      0  53.1000     55         

# Gaussian Naïve Bayes Classifier
 - 80/20 training-testing split

In [6]:
X_train,X_Test,Y_train, Y_Test = train_test_split(X,Y, random_state = 1, test_size = .20)

from sklearn.naive_bayes import GaussianNB

NB = GaussianNB()
NB.fit(X_train, np.asarray(Y_train))
y_pred = NB.predict(X_Test)
scores = cross_validate(NB, X, Y,scoring = ('accuracy','neg_mean_squared_error'), cv = 10)


In [7]:
print("Accuracy:",NB.score(X_Test,Y_Test))
print("MSE:", mean_squared_error(Y_Test,y_pred))

Accuracy: 0.7430167597765364
MSE: 0.2569832402234637


In [8]:
#print(scores['test_accuracy'])
print("Accuracy:",scores['test_accuracy'].mean())
print("MSE:",np.mean(np.abs(scores['test_neg_mean_squared_error'])))

Accuracy: 0.7845193508114856
MSE: 0.2154806491885144
